## 1. Progress Report (Student’s Voice)

- **DICOM CT loading & header parsing**  
  “I successfully loaded 207 DICOM slices in about 0.4 s, with voxel spacing (0.78125, 0.78125, 2.5 mm). The missing `AcquisitionNumber` tags triggered warnings—so far it doesn’t break anything, but I should consider a fallback (e.g., `InstanceNumber`) if I use acquisition numbers downstream.”

- **Multi-ROI segmentation import & validation**  
  “The segmentation loader found only segment 1 (20 347 voxels) and flagged segments 2–4 as missing. I need to double-check my DICOM-RTSTRUCT input or update the loader to gracefully handle partial ROI sets.”

- **Slice overlay visualization**  
  “Overlay for slice 23 looks good: CT in grayscale, tumor mask in semi-transparent jet, and green contours outline the ROI precisely. Saved as `slice_overlay_23.png`. I might want to try a single colormap plus contour color to reduce potential color-map bias.”

- **Maximum Intensity Projections (MIPs)**  
  “Computed coronal and sagittal MIPs in ~0.1 s and saved `mips_combined.png`. The tumor silhouette is clear, but I wonder if I should apply CT windowing (e.g., soft-tissue window) before projection to improve contrast.”

- **Rotating 3D GIF animation**  
  “Generated an 18-frame rotation GIF at 10 fps in ~0.04 s; file size is ~2 MB. It’s visually effective, but perhaps exporting as MP4 or increasing frame count (e.g., 36 frames) would give smoother rotation without bloating file size.”

- **Performance benchmarking & logging**  
  “All timing and memory metrics (CT load, segmentation load, MIP compute, GIF creation; memory peaks ~473 MB) were logged to `performance_metrics.csv`. Next, I could plot these metrics to quickly identify any bottlenecks.”

---

## 2. Student’s Open Questions

1. **Missing DICOM tags**  
   - Why might `AcquisitionNumber` be missing in my CT series? Should I fallback to another tag (e.g. `InstanceNumber`), or is it better to require a consistent tag for production code?

2. **Partial ROI segments**  
   - Only segment 1 was loaded; segments 2–4 are missing. Is this a dataset issue or should I modify `load_segmentation` to handle multiple segment sets differently?

3. **Isotropic resampling**  
   - Given anisotropic voxel spacing (2.5 mm slice thickness), should I resample to isotropic voxels for more accurate MIP and 3D rotations?

4. **Visualization performance**  
   - On large volumes, per-frame rotation + MIP is slow. Are there GPU-accelerated or vectorized libraries (e.g., VTK, napari, OpenGL) I could use to improve interactivity?

5. **Reproducible environment**  
   - Do I need to capture exact library versions (via `pip freeze` or `environment.yml`) and seed any random operations to guarantee that anyone rerunning this notebook gets identical results?

6. **Advanced profiling**  
   - Would integrating `cProfile`, `line_profiler`, or `pytest-benchmark` give me finer-grained insights than the current manual timing/logging approach?

---

## 3. Cell-by-Cell Student Commentary

### **TITLE & Objective**
```markdown
# TITLE
## Objective
1. Load DICOM CT images and segmentations.
2. Visualize data and check alignment.
3. Create a MIP animation with tumor mask overlay.
```
- *Why:* Sets the stage and reminds me of the three core tasks before diving in.

---

### **Imports & Auto-Reload**
```python
%load_ext autoreload
%autoreload 2

import os, csv, time, logging
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import psutil
from skimage.measure import find_contours

from utils import (
    load_ct_slices, load_segmentation,
    create_mip, overlay_mask,
    rotate_volume, LOG_CSV
)
```
- *What & Why:*  
  - `%autoreload` lets me edit `utils.py` and see changes immediately.  
  - I import all needed libraries for I/O, timing, logging, plotting, and image processing.

- *Fit:* Prepares the environment so that utility functions are always up to date and all dependencies are loaded.

---

### **Setup & Logging Configuration**
```python
%matplotlib inline
plt.style.use('grayscale')

OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

root = logging.getLogger()
root.handlers.clear()
root.setLevel(logging.INFO)
# Console handler
console_h = logging.StreamHandler()
console_h.setFormatter(logging.Formatter('[%(levelname)s] %(message)s'))
root.addHandler(console_h)
# CSV handler
file_h = logging.FileHandler(LOG_CSV, mode='a')
file_h.setFormatter(logging.Formatter('%(asctime)s,%(levelname)s,%(name)s,%(message)s'))
root.addHandler(file_h)

logger = logging.getLogger(__name__)
```
- *Comment:*  
  - Inline grayscale is appropriate for medical images.  
  - Logging both to console and CSV ensures I don’t lose performance data.

- *Question:*  
  - Will appending to the same CSV across notebook reruns duplicate entries? Should I clear the file at each run?

---

### **Load and Benchmark CT**
```python
logger.info("Starting CT load...")
start_time = time.perf_counter()

ct_volume, metadata, internal_times = load_ct_slices(CT_DIR)

elapsed = time.perf_counter() - start_time
metrics.update(internal_times)
metrics['total_ct_load_s'] = elapsed
metrics['memory_after_ct_load_bytes'] = process.memory_info().rss

logger.info(f"CT shape: {ct_volume.shape}, spacing: {metadata['spacing']}")
```
- *Comment:*  
  - I load 207 slices in ~0.4 s; metadata shows spacing (0.78125, 0.78125, 2.5).  
- *Question:*  
  - That 2.5 mm slice thickness is much larger than in-plane resolution—do I need to resample before further processing?

---

### **Load and Benchmark Segmentation**
```python
logger.info("Starting segmentation load...")
start_time = time.perf_counter()

mask_volume = load_segmentation(SEG_PATH, metadata['positions'])

elapsed = time.perf_counter() - start_time
metrics['segmentation_load_s'] = elapsed
metrics['memory_after_seg_bytes'] = process.memory_info().rss

logger.info(f"Segment 1 voxel count: {np.sum(mask_volume==1)}")
```
- *Comment:*  
  - Segmentation load was fast (~0.05 s), but only segment 1 is present.  
- *Question:*  
  - Should the loader throw an error or simply warn when expected segments are missing?

---

### **Data Verification (Slice Overlay)**
```python
tumor_slices = np.where(mask_volume.sum(axis=(1,2)) > 0)[0]
first_slice_idx = tumor_slices[0]
ct_slice = ct_volume[first_slice_idx]
mask_slice = mask_volume[first_slice_idx]

fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].imshow(ct_slice); axes[0].axis('off')
axes[1].imshow(ct_slice)
masked = np.ma.masked_where(mask_slice==0, mask_slice)
axes[1].imshow(masked, cmap='jet', alpha=0.5)
for cnt in find_contours(mask_slice,0.5):
    axes[1].plot(cnt[:,1],cnt[:,0],color='lime',linewidth=1.5)
fig.savefig(os.path.join(OUTPUT_DIR,f"slice_overlay_{first_slice_idx}.png"))
plt.show()
```
- *Comment:*  
  - Overlay looks correct; saved as `slice_overlay_23.png`.  
- *Questions:*  
  - Would a single consistent colormap (e.g., grayscale + colored contour) improve clarity?  
  - Should I loop over all slices with ROIs and generate a montage?

---

### **Maximum Intensity Projections**
```python
logger.info("Computing MIPs...")
start_time = time.perf_counter()

mip_coronal = create_mip(ct_volume, axis=1)
mip_sagittal = create_mip(ct_volume, axis=0)

metrics['mip_compute_s'] = time.perf_counter() - start_time
metrics['memory_after_mip_bytes'] = process.memory_info().rss

fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0].imshow(mip_coronal); axes[0].axis('off')
axes[1].imshow(mip_sagittal); axes[1].axis('off')
fig.savefig(os.path.join(OUTPUT_DIR,"mips_combined.png"))
plt.show()
```
- *Comment:*  
  - MIPs generated quickly; images clearly show the tumor’s outline.  
- *Question:*  
  - Should I adjust window/level before MIP to highlight soft-tissue contrast?

---

### **Rotating MIP Animation**
```python
n_frames, fps = 18, 10
gif_path = os.path.join(OUTPUT_DIR,'mip_rotation_simple.gif')
fig = plt.figure(figsize=(6,6))

def update(frame_idx):
    angle = 360 * frame_idx / n_frames
    vol_rot  = rotate_volume(ct_volume, angle, axis='y')
    mask_rot = rotate_volume(mask_volume.astype(np.uint8), angle, axis='y')
    mip_ct   = create_mip(vol_rot, axis=1)
    mip_mask = create_mip(mask_rot, axis=1)
    img = overlay_mask(mip_ct, mip_mask)
    plt.clf(); plt.axis('off'); plt.title(f"{angle:.0f}°")
    plt.imshow(img)

anim = FuncAnimation(fig, update, frames=n_frames)
anim.save(gif_path, writer=PillowWriter(fps=fps))
```
- *Comment:*  
  - Animation is smooth; each frame rotates 20° and overlays the mask.  
- *Questions:*  
  - Would an MP4 be preferable for smaller size or embedding?  
  - Should I precompute rotations in parallel to speed up GIF creation for larger volumes?

---

### **Export Performance Metrics**
```python
perf_csv = os.path.join(OUTPUT_DIR,"performance_metrics.csv")
with open(perf_csv,'w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['metric','value','unit'])
    for k,v in metrics.items():
        writer.writerow([k,v,unit_map[k]])
```
- *Comment:*  
  - I now have a CSV with all timings and memory usage.  
- *Question:*  
  - Next step: plot these metrics (bar chart, memory vs. time) to visually identify any bottlenecks?